In [318]:
# Reload all modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [319]:
import re
import json
import glob
from urllib.parse import urljoin
import requests
from pathlib import Path
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import src.data.parse_html as parse_html
import src.data.metadata as metadata
import src.data.text_process as text_process

from dotenv import load_dotenv
load_dotenv()

True

In [8]:
proj_path = Path(os.environ['PROJ_DIR'])
data_dir = proj_path / 'data'
interim_data = data_dir / 'interim'
raw_data = data_dir / 'raw'
letters_path = data_dir / 'external' / 'Cartas-txt'
legal_docs_path = data_dir / 'external' / 'documentos arreglados CorpMA'

In [4]:
meta_fields = ['meta_id', 'format', 'corpus', 'unknown_id', 'date (place)', 'doc_type', 'abstract', 'author']
es_months = ['enero', 'febrero' ,'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
es_month_name_to_number = {name: str(i+1).zfill(2) for i, name in enumerate(es_months)}
# add user agent to trick website into thinking we're accessing from a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [5]:
corpus_metadata = {
    'corpusmallorca':
        {
            'base_url': 'https://corpusmallorca.es',
            'remote_doc_list': 'buscar.php?&paleografica=on&critica=on&maxFiles=999999&nFilei=0&nFilef=1000&selDonde=0',
        },
    'corpuscodea':
        {
            'base_url': 'https://corpuscodea.es/corpus',
            'remote_doc_list': 'consultas.php',
            'local_doc_list': 'docs.html'
        },
    'corpuscharta':
        {
            'base_url': 'https://www.corpuscharta.es',
            'remote_doc_list': 'consultas.html',
            'local_doc_list': 'docs.html',
        },
}
doc_url_patt = "{corpus_base_url}/documento.php"

In [275]:
field_norm = {
    'IDENTIFICADOR': 'meta_id',
    'REGESTO': 'abstract',
    'DOCUMENTO': 'abstract',
    'PAÍS': 'country',
    'PROVINCIA': 'region',
    'POBLACIÓN': 'locality',
    'FECHA': 'year',
    'SIGLO': 'century',
    'TIPOLOGÍA': 'doc_type',
    'TIP. DOCUMENTAL': 'doc_type',
    'TIP. DIPLOMÁTICA': 'diplo_type', # TODO: ??
    'ARCHIVO (SIGN.)': 'archive',
    'PALABRAS': 'nr_words',
    'MUJER': 'woman',
    'LETRA': 'writing',
    'ÁMBITO': 'context',
    'CLAVE': 'keywords',
    'COPISTA (FÓRM.)': 'copyist',
    'DOCUMENTO': 'meta_id',
    'ARCHIVO': 'archive',
    'AÑO': 'year',
    'LUGAR': 'locality',
}

In [7]:
corpus_name = 'corpusmallorca'
# corpus_name = 'corpuscodea'
corpus_dict = corpus_metadata[corpus_name]
corpus_dir = interim_data / corpus_name
corpus_dir.mkdir(exist_ok=True, parents=True)
docs_url = doc_url_patt.format(corpus_base_url=corpus_dict['base_url'])

# Extract data from corpus

## Online 

In [ ]:
if 'local_doc_list' in corpus_dict:
    doc_list_path = data_dir / 'external' / corpus_name / corpus_dict['local_doc_list']
    with open(doc_list_path, 'r') as f:
        doc_list_html = f.read()
else:
    doc_list_url = urljoin(corpus_dict['base_url'], corpus_dict['remote_doc_list'])
    doc_list_html = requests.get(doc_list_url, headers=headers).content
doc_list_soup = BeautifulSoup(doc_list_html, 'html.parser')

In [ ]:
if corpus_name == 'corpusmallorca':
    doc_ids = []
    for link in doc_list_soup.find_all('a'):
        doc_ids.append(re.match("javascript:abrirDocumento\('(.*)'\)", link.get('href')).groups()[0])

    for doc_id in tqdm(doc_ids):
        response = requests.get(docs_url, headers=headers, params={'documento': doc_id, 'paleografica': 'on', 'critica': 'on'})
        soup = BeautifulSoup(response.content, 'html.parser')

        data = parse_html.extract_metadata(soup, meta_fields, es_month_name_to_number)
        
        raw_text_soups = soup.find_all(class_='textopaleo')
        data['raw_text'] = ''.join([str(s) for s in raw_text_soups])
        data['text'] = text_process.clean(parse_html.extract_text(raw_text_soups))
        
        with open(corpus_dir / f'{doc_id}.json', 'w') as f:
            json.dump(data, f)
        print(f'{doc_id} done')

else:
    # field_heads = soup.find(id='Tabla_Inventario').thead.find_all('tr')[-1].find_all('th')
    field_heads = doc_list_soup.find(id='Tabla_Inventario_wrapper').find('thead').find_all('tr')[-1] # codea
    ordered_field_names = [field_norm[h.div.text] for h in field_heads]
    pbar = tqdm(doc_list_soup.find(id='Tabla_Inventario').tbody.find_all('tr'))
    for doc_meta in pbar:
        field_values = [f.text.strip() for f in doc_meta.find_all('td')]
        data = dict(zip(ordered_field_names, field_values))
        # roman.fromRoman(data['century'])
        doc_id = data['meta_id']
        pbar.set_description(doc_id)
        if doc_id in docs_to_retrieve: ###
            response = requests.get(docs_url, headers=headers, params={'documento': doc_id, 'paleografica': 'on', 'critica': 'on'})
            soup = BeautifulSoup(response.content, 'html.parser')
            raw_text_soups = soup.find_all(class_='textopaleo')
            data['raw_text'] = ''.join([str(s) for s in raw_text_soups])
            data['text'] = text_process.clean(parse_html.extract_text(raw_text_soups))
            with open(corpus_dir / f'{doc_id}.json', 'w') as f:
                json.dump(data, f)

CODEA-2495: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [02:10<00:00, 19.12it/s]


In [ ]:
response = requests.get(docs_url, headers=headers, params={'documento': 'CODEA-1774', 'paleografica': 'on', 'critica': 'on'})
soup = BeautifulSoup(response.content, 'html.parser')
raw_text_soups = soup.find_all(class_='textopaleo')
# data['raw_text'] = ''.join([str(s) for s in raw_text_soups])
text = parse_html.extract_text(raw_text_soups)

CODCAR-0184 duplicated in charta's inventory for some reason

## Offline

In [269]:
import src.data.parse_docs as parse_docs

In [307]:
corpus_name = 'corpusmallorca'
corpus_dir = interim_data / corpus_name
corpus_src_dir = raw_data / corpus_name
already_in = [p.stem for p in corpus_dir.glob('*.json')]
files_to_process = [
    path.name for path in corpus_src_dir.iterdir()
    if  path.suffix == '.docx'
    # if path.stem not in already_in and path.suffix == '.docx'
]
for fname in tqdm(files_to_process):
    path = corpus_src_dir / fname
    # have to register this because there are a sizable number of mismatches
    data = parse_docs.parse(path, field_norm)
    raw_text = data.get('raw_text', '')
    data['text'] = text_process.clean(raw_text, space_after_newline=False)
    # saving with same file name because this has to be unique, since files are stored
    # in same directory. Not necessarily true of given meta_id, which are inconsistent.
    with open(corpus_dir / f'{doc_id}.json', 'w') as f:
        json.dump(data, f)
    # else:
    #     print(f"The ID provided in the header of {fname} doesn't match: {doc_id}")
    # d = read_docx(path, corpus_dir)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 736/736 [00:19<00:00, 38.17it/s]


In [308]:
import pandas as pd

In [309]:
records = []
for fname in files_to_process:
    path = corpus_dir / f'{fname[:-5]}.json'
    with open(path) as f:
        records.append(json.load(f))
auto_df = pd.DataFrame.from_records(records)

In [310]:
auto_df.loc[auto_df['file_id'] != auto_df['meta_id']]

,file_id,meta_id,archive,year,locality,region,country,abstract,doc_type,raw_text,text
17,AMDE38,AMDE_38,Arxiu Municipal de Deià,1868,Deià,Islas Baleares,España,Petición al director de caminos vecinales de q...,informes y relaciones,{h 1r} M<uy> Y<lustre> S<eñor>\nHabiendo manif...,Muy Ylustre Señor\nHabiendo manifestado a est...
35,AA035_01,AA035_00,Arxiu del Regne de Mallorca,1804,Bunyola,Islas Baleares,España,Robo cometido en Orient por tres hombres que p...,Informes y relaciones,{h 0r} S<umari>a nº 8 1804\nExpediente relativ...,Sumaria nº 8 1804\nExpediente relativo à la C...
62,AMSL23_02,AMSL23,Arxiu Municipal de Selva,1894,Selva,Islas Baleares,España,Instrucción de la causa contra Juan Vives por ...,Actas y declaraciones,{h 23r} Providencia Selva cinco de Abril de 18...,Providencia Selva cinco de Abril de 1894.\nRe...
92,AA013_10,AA013_010,Arxiu del Regne de Mallorca,1784,Palma,Islas Baleares,España,Copia de un oficio del ministro de Marina en e...,Informes y relaciones,{h 21r} |...| JHS\nPalma 28. Sett<iemb>re de 1...,|...| JHS\nPalma 28. Settiembre de 1784.\nEn ...
134,APB02,APB_02 ANDRÉS,Archivo Histórico de Señales Marítimas de la A...,1855,Palma,Illes Balears,España,La autoridad portuaria informa del envío de gu...,Cartas oficiales,{h 1r} [margen: Sanidad | Enterado] Debiendo t...,Debiendo trasladarse los\nbuques cuarentenar...
176,AMSL26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,AA004_02,NaN,Arxiu del Regne de Mallorca,1760,Palma,Islas Baleares,España,"Declaración de Jorge Ramis, el cual narra todo...",actas y declaraciones,{h 2r} Dia 6 Novi<em>bre 1760.\n[margen: Tes<t...,Dia 6 Noviembre 1760.\n Jorge Ramis natural d...
185,AHNI04_01,AHNI04,Archivo Histórico Nacional,s.a [ca.1802],Palma,Islas Baleares,España,Alegación fiscal del proceso de fe de Isabel S...,Actas y declaraciones,{h 1r} El Ynq<uisidor> Fiscal interino de Mall...,El Ynquisidor Fiscal interino de Mallorca con...
187,AMCV01_03,AMCV,Arxiu Municipal de Calvià,1723,Palma,Islas Baleares,España,"Josep Antonio de Chaves Osorio, teniente gener...",Informes y relaciones,{h 1r} D<o>n Joseph Antonio de Xaves Ossorio T...,Don Joseph Antonio de Xaves Ossorio Teniente ...
200,AA002_05,AA002_5,Arxiu del Regne de Mallorca,1720,Sineu,Islas Baleares,España,Entrega del lugarteniente de Sineu Antonio Roi...,Actas y declaraciones,{h 11r} |...| [margen: Auto] En la Villa de Si...,"|...| En la Villa de Sineu en veinte, y nuev..."


differences are mostly slight, rather insignificant ones, and sometimes wrong format / no metadata at all

In [313]:
auto_df.loc[452, 'text']

'  | Excelentisimo Señor. | Haviendo llamado los Au|tos que seciguen por esta | Auditoria contra | Bartholome Solivellas | Sargento de las milicias | Vrbanas he visto que | se halla acusado de | mal manejo en distin|tas recaudaciones que | estubieron â su cargo | en la villa de Selva, | cuya causa recibida | á prueba se halla | en estado de alegacion. | Y no quedando indenni|zado de dicha acusacion | parece muy impropio | el que pendiente la mis|ma se le encargen, y | confien otras recauda|ciones de aquel Pueblo. | y de consiguiente me | parece fundada la | solicitud de los recur|rentes. Palma 10 de | Marzo de 1817. | Excelentisimo Señor. | Palet.] Excelentisimo Señor\nDon Pedro Francisco Morro de Vaxella Subdelegado de \nMarina. Don Bartholome Amer del castell theniente de Vrbanos, el\nDoctor en Medicina Don Juan Tony, Don Bartholome Vidal \nSubdelegado de la Real Intendencia, y el Doctor Don Miguel Ferragut\nMedico, todos vecinos de la Villa de Selva llevados del mas\ncincero amor al

# Edit pre-processed data

In [ ]:
for path in corpus_dir.glob('*.json'):
    with open(path, 'r') as f:
        data = json.load(f)
        data['year'] = data.pop('date', data.get('year'))
        data['doc_type'] = data.pop('doct_type', data.get('doc_type'))
    with open(path, 'w') as f:
        json.dump(data, f)